<a href="https://colab.research.google.com/github/izzamokhtar/urbanrent/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

!ngrok authtoken 2q3MywynEFtR5FkvEPDc4HIEvHz_ky4NtHhoZD952M4HGaNK # Replace <your_auth_token> with your actual token
!streamlit run app.py &> /dev/null &
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://e1c4-35-190-150-132.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pkill -f ngrok

In [ ]:
#yang ni latest
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings

st.set_page_config(page_title="Rent Price Prediction", page_icon=":heavy_dollar_sign:")
filterwarnings('ignore', category=FutureWarning)

# Sidebar for navigation
#page = st.sidebar.radio("Navigate", ["Rental Prediction", "Interactive Graphs", "Data Analysis"])
# Sidebar navigation with icons
page = st.sidebar.radio(
    "",
    [
        "💲 Rent Prediction",
        "📊 Interactive Graphs",
        "📈 Map Visualisation"
    ]
)

if page == "💲 Rent Prediction":
    st.empty()
    exec(open("rental.py").read())

if page == "📊 Interactive Graphs":
    st.empty()
    exec(open("dashboard.py").read())

if page == "📈 Map Visualisation":
    st.empty()
    exec(open("visual.py").read())

Overwriting app.py


In [ ]:
#latest
%%writefile rental.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings

# Load the best model
best_model_pipeline = joblib.load("/content/drive/MyDrive/FYP B IZZA/best_model_pipeline.pkl")
linear_model_pipeline = joblib.load('/content/drive/MyDrive/FYP B IZZA/linear_regression_pipeline.pkl')
#random_forest_pipeline = joblib.load('/content/drive/MyDrive/FYP B IZZA/random_forest_pipeline.pkl')
gradient_boosting_pipeline = joblib.load('/content/drive/MyDrive/FYP B IZZA/gradient_boosting_pipeline.pkl')

# Load the best model's name
with open("/content/drive/MyDrive/FYP B IZZA/best_model_name.txt", "r") as f:
    best_model_name = f.read()

# Extract categories for user input from the best model's preprocessor
preprocessor = best_model_pipeline.named_steps['preprocessor']
onehot_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
location_categories = onehot_encoder.categories_[0]
property_type_categories = onehot_encoder.categories_[1]

st.title("Rental Price Prediction")
st.write(f"Predict monthly rental prices using the most accurate model: **{best_model_name}**")

# User input
location = st.selectbox("Location", location_categories)
property_type = st.selectbox("Property Type", property_type_categories)
rooms = st.number_input("Number of Rooms", value=2)
bathrooms = st.number_input("Number of Bathrooms", value=1)
parking = st.number_input("Number of Parking Spaces", value=1)

# Combine user inputs into a DataFrame
user_input = pd.DataFrame({
    'location': [location],
    'property_type': [property_type],
    'rooms': [rooms],
    'bathroom': [bathrooms],
    'parking': [parking]
})

# Predict with the best model
if st.button("Predict"):
    prediction = best_model_pipeline.predict(user_input)[0]
    st.write(f"Predicted Rent using {best_model_name}: RM{prediction:.2f}")
    linear_prediction = linear_model_pipeline.predict(user_input)[0]
    gradient_boosting_prediction = gradient_boosting_pipeline.predict(user_input)[0]

    # Plot predictions
    algorithms = ['Linear Regression', 'Random Forest', 'Gradient Boosting']
    predictions = [linear_prediction, prediction, gradient_boosting_prediction]

    # Create a bar chart
    fig, ax = plt.subplots()
    ax.bar(algorithms, predictions, color=['blue', 'green', 'orange'])
    ax.set_ylabel('Predicted Rental Price (RM)')
    ax.set_title('Comparison of Predicted Prices')
    st.pyplot(fig)

Writing rental.py


In [ ]:
#latesttt
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import joblib
import numpy as np
from itertools import product

st.title("Rent Pricing at Kuala Lumpur and Selangor")

# Load the dataset
@st.cache_data
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')
    df.columns = df.columns.str.strip().str.lower()
    df['monthly_rent'] = df['monthly_rent'].str.replace('RM', '').str.replace('per month', '').str.replace(' ', '')
    df['monthly_rent'] = pd.to_numeric(df['monthly_rent'], errors='coerce')
    df = df.dropna(subset=['monthly_rent'])
    return df

df = load_data()

# Load the trained model pipeline
@st.cache_resource
def load_model():
    model = joblib.load('/content/drive/MyDrive/FYP B IZZA/best_model_pipeline.pkl')  # Your trained model
    return model

model = load_model()

# Sidebar for user input (only Location and Property Type)
st.sidebar.title('Interactive Graphs')
st.sidebar.header('Graph Parameters')

# User selects the parameters for filtering data
selected_locations = st.sidebar.multiselect('Select Locations', df['location'].unique())
selected_property_types = st.sidebar.multiselect('Select Property Types', df['property_type'].unique())

# Ensure there is at least one location and one property type selected
if not selected_locations or not selected_property_types:
    st.warning("Please select at least one location and one property type.")
    st.stop()

# Generate all combinations of selected locations and property types
combinations = list(product(selected_locations, selected_property_types))

# Prepare input for the model
model_input = pd.DataFrame(combinations, columns=['location', 'property_type'])

# Add dummy values for other features (rooms, bathroom, parking)
model_input['rooms'] = 2  # Default room value
model_input['bathroom'] = 1  # Default bathroom value
model_input['parking'] = 1  # Default parking value

# Apply the same transformations as the model expects (e.g., one-hot encoding, scaling)
# Assuming the model uses a ColumnTransformer for preprocessing:
try:
    transformed_input = model.transform(model_input)
except AttributeError:
    # Handle the case where the model is not a pipeline and does not need transformation
    transformed_input = model_input  # You may need to manually apply transformations here

# Get predictions for rent prices from the model
predicted_prices = model.predict(transformed_input)

# Prepare the data for the graph (create a DataFrame with predicted prices)
graph_data = pd.DataFrame({
    'location': model_input['location'],
    'property_type': model_input['property_type'],
    'predicted_rent': predicted_prices
})

# Round the predicted prices to 2 decimal points
graph_data['predicted_rent'] = graph_data['predicted_rent'].round(2)

# Visualization Columns
col1, col2 = st.columns(2)

# Graph 1: Average Price by Location (based on model prediction)
avg_price_location = graph_data.groupby('location')['predicted_rent'].mean().reset_index()
avg_price_location['predicted_rent'] = avg_price_location['predicted_rent'].round(2)
fig1 = px.bar(avg_price_location, x='location', y='predicted_rent', color='location', title="Average Price by Location")

# Increase the graph size and rename axis labels
fig1.update_layout(
    title_font_size=20,  # Adjust title font size
    xaxis_title='Location',  # Rename the x-axis
    yaxis_title='Price (RM)',  # Rename the y-axis
    xaxis_title_font_size=14,  # Adjust x-axis title font size
    yaxis_title_font_size=14,  # Adjust y-axis title font size
    showlegend=True,  # Show legend if necessary
)

col1.plotly_chart(fig1, use_container_width=True)

# Graph 2: Average Price by Property Type (based on model prediction)
avg_price_property = graph_data.groupby('property_type')['predicted_rent'].mean().reset_index()
avg_price_property['predicted_rent'] = avg_price_property['predicted_rent'].round(2)
fig2 = px.bar(avg_price_property, x='property_type', y='predicted_rent', color='property_type', title="Average Price by Property Type")

# Increase the graph size and rename axis labels
fig2.update_layout(
    title_font_size=20,  # Adjust title font size
    xaxis_title='Property Type',  # Rename the x-axis
    yaxis_title='Price (RM)',  # Rename the y-axis
    xaxis_title_font_size=14,  # Adjust x-axis title font size
    yaxis_title_font_size=14,  # Adjust y-axis title font size
    showlegend=True,  # Show legend if necessary
)

col2.plotly_chart(fig2, use_container_width=True)


Overwriting dashboard.py


In [ ]:
pip install geopy

In [ ]:
%%writefile visual.py
import streamlit as st
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

# Load your dataset
@st.cache_data
def load_data():
    # Replace with your actual dataset path
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')

    # Clean dataset columns (normalize column names)
    df.columns = df.columns.str.strip().str.lower()

    # Clean 'monthly_rent' column (remove RM and convert to numeric)
    df['monthly_rent'] = (
        df['monthly_rent']
        .str.replace('RM', '', regex=False)
        .str.replace('per month', '', regex=False)
        .str.replace(' ', '', regex=False)
        .apply(pd.to_numeric, errors='coerce')
    )

    # Drop rows with NaN values in important columns
    df = df.dropna(subset=['monthly_rent', 'prop_name', 'location', 'property_type'])

    return df

df = load_data()

# Geocoding function with user-agent
def geocode_location(location_name):
    geolocator = Nominatim(user_agent="my_rent_dashboard (213061@student.upm.edu.my)")  # Change this to your app name and contact
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Sidebar for user input
st.sidebar.title('Interactive Map for Locations')
st.sidebar.header('Select Location and Property Type')

# Get unique locations and property types from the dataset
locations = df['location'].unique()
property_types = df['property_type'].unique()

# User selects the parameters for filtering data
selected_locations = st.sidebar.multiselect('Select Locations', locations)
selected_property_types = st.sidebar.multiselect('Select Property Types', property_types)

# Filter the dataset based on the selected location and property type
filtered_data = df[
    (df['location'].isin(selected_locations)) &
    (df['property_type'].isin(selected_property_types))
]

# Check if any data is available after filtering
if filtered_data.empty:
    st.warning("No data available for the selected filters.")
else:
    # Initialize map centered at a default location (e.g., Kuala Lumpur)
    m = folium.Map(location=[3.139, 101.6869], zoom_start=12)
    marker_cluster = MarkerCluster().add_to(m)

    # Iterate through the filtered data to add markers for each property
    for index, row in filtered_data.iterrows():
        prop_name = row['prop_name']
        location_name = row['location']

        # Geocode location name to get latitude and longitude
        lat, lon = geocode_location(location_name)

        if lat and lon:
            folium.Marker(
                location=[lat, lon],
                popup=f"Property: {prop_name}<br>Location: {location_name}<br>Type: {row['property_type']}<br>Price: RM {row['monthly_rent']}",
            ).add_to(marker_cluster)

    # Display the map
    st.write("### Property Locations Map")
    folium_static(m)

Overwriting visual.py


In [ ]:
%%writefile visual.py
import streamlit as st
import pandas as pd
import plotly.express as px
import pickle
from geopy.geocoders import Nominatim
import joblib

st.title("Interactive Map Visualization")

# 1. Load the trained model
@st.cache_resource
def load_model():
    with open('/content/drive/MyDrive/FYP B IZZA/best_model_pipeline.pkl', 'rb') as file:  # Update path to your trained model
        model = pickle.load(file)
    return model

model = load_model()

# 2. Assuming your original dataset (the one you used for training the model) is available
# You can load the original dataset to extract unique locations and property types

@st.cache_data
def load_training_data():
    # Load the dataset used to train the model
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')  # Update with your dataset path
    return df

df = load_training_data()

# 3. Extract unique locations and property types from the training data (not hardcoded)
locations = df['location'].unique().tolist()
property_types = df['property_type'].unique().tolist()

# 4. Sidebar for User Input
st.sidebar.header("Select Map Parameters")
selected_location = st.sidebar.selectbox("Select Location", locations)
selected_property_type = st.sidebar.selectbox("Select Property Type", property_types)

# 5. Geocode the selected location to get latitude and longitude
@st.cache_data
def geocode_location(location_name):
    geolocator = Nominatim(user_agent="geoapi")
    try:
        location = geolocator.geocode(location_name)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

latitude, longitude = geocode_location(selected_location)

# 6. Display Interactive Map
if latitude and longitude:
    st.header("Map Visualization")
    # Create a DataFrame for Plotly
    map_data = pd.DataFrame({
        "location": [selected_location],
        "property_type": [selected_property_type],
        "latitude": [latitude],
        "longitude": [longitude]
    })

    # Interactive Map using Plotly
    fig = px.scatter_mapbox(
        map_data,
        lat="latitude",
        lon="longitude",
        hover_name="location",
        hover_data=["property_type"],
        color_discrete_sequence=["blue"],
        zoom=12,
        height=700,
        title="Selected Location on Map"
    )
    fig.update_layout(mapbox_style="open-street-map")
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("Could not find latitude and longitude for the selected location. Please try another location.")


Overwriting visual.py


In [ ]:
%%writefile visual.py
import streamlit as st
import pandas as pd
import plotly.express as px
from geopy.geocoders import Nominatim
import time

st.title("Location Map Visualization Without Latitude/Longitude")

# 1. Load Dataset
@st.cache_data
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')  # Replace with your dataset
    df.columns = df.columns.str.strip().str.lower()
    return df

df = load_data()

# 2. Sidebar inputs for filtering
st.sidebar.header("Graph Parameters")
selected_property_type = st.sidebar.selectbox("Select Property Type", df['property_type'].unique())

# 3. Filter dataset based on user input
filtered_data = df[df['property_type'] == selected_property_type]

# 4. Geocoding Function (Get latitude & longitude for locations)
@st.cache_data
def geocode_location(location_names):
    geolocator = Nominatim(user_agent="geoapi")  # Initialize geolocator
    lat_lon_data = []
    for location in location_names:
        try:
            location_info = geolocator.geocode(location)  # Fetch location details
            if location_info:
                lat_lon_data.append((location, location_info.latitude, location_info.longitude))
            else:
                lat_lon_data.append((location, None, None))  # Append None if not found
            time.sleep(1)  # To respect rate limits
        except Exception as e:
            lat_lon_data.append((location, None, None))
    return pd.DataFrame(lat_lon_data, columns=["location", "latitude", "longitude"])

# 5. Get Latitude/Longitude for Unique Locations
unique_locations = filtered_data['location'].unique()
location_data = geocode_location(unique_locations)

# 6. Merge Latitude/Longitude Back to Dataset
merged_data = pd.merge(filtered_data, location_data, on="location", how="left")

# 7. Check for Missing Coordinates
merged_data = merged_data.dropna(subset=['latitude', 'longitude'])

# 8. Plot Map
if not merged_data.empty:
    st.header("Map Visualization of Locations")
    fig = px.scatter_mapbox(
        merged_data,
        lat="latitude",
        lon="longitude",
        hover_name="location",
        zoom=8,
        height=600,
        title="Map Visualization of Selected Property Type"
    )
    fig.update_layout(mapbox_style="open-street-map")
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("No valid locations found for mapping.")


Overwriting visual.py


In [ ]:
%%writefile dashboard.py
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import pickle

st.title("Rent Pricing at Kuala Lumpur and Selangor")

# Load dataset dynamically
# Replace 'your_dataset.csv' with the actual file path to your dataset
@st.cache_data
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')  # Replace with your dataset file
    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()

    # Clean 'monthly_rent' column (removes RM, per month, and spaces)
    df['monthly_rent'] = (
        df['monthly_rent']
        .str.replace('RM', '', regex=False)           # Remove RM
        .str.replace('per month', '', regex=False)    # Remove 'per month'
        .str.replace(' ', '', regex=False)            # Remove spaces between numbers
        .apply(pd.to_numeric, errors='coerce')         # Convert to float, coerce errors to NaN
    )

    # Ensure numeric columns are properly converted to numeric, coercing errors to NaN
    numeric_columns = ['rooms', 'bathroom', 'parking']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Use pd.to_numeric to handle non-numeric values

    # Drop rows with NaN values in numeric columns if necessary (e.g., for prediction)
    df = df.dropna(subset=numeric_columns + ['monthly_rent'])

    return df

df = load_data()

# Check if required columns exist
required_columns = ['location', 'property_type', 'rooms', 'bathroom', 'parking', 'monthly_rent']
if not all(col in df.columns for col in required_columns):
    st.error(f"The dataset must contain the following columns: {required_columns}")
    st.stop()

# Convert 'monthly_rent' column to numeric, forcing errors to NaN (invalid data will be excluded)
df['monthly_rent'] = pd.to_numeric(df['monthly_rent'], errors='coerce')

# Sidebar for user input (only for graph parameters)
st.sidebar.title('Interactive Graphs')
st.sidebar.header('Graph Parameters')

# User selects the parameters for filtering data
selected_locations = st.sidebar.multiselect('Select Locations', df['location'].unique())
selected_property_types = st.sidebar.multiselect('Select Property Types', df['property_type'].unique())

# Filter dataset based on user selection
filtered_data = df[
    (df['location'].isin(selected_locations)) &
    (df['property_type'].isin(selected_property_types))
]

# Check if the filtered data is empty
if filtered_data.empty:
    st.warning("No data available for the selected filters.")
else:
    # Visualization Columns
    col1, col2 = st.columns(2)

    # Graph 1: Average Price by Location
    avg_price_location = filtered_data.groupby('location')['monthly_rent'].mean().reset_index()
    avg_price_location['monthly_rent'] = avg_price_location['monthly_rent'].round(2)  # Round to 2 decimal places
    fig1 = px.bar(avg_price_location, x='location', y='monthly_rent', color='location', title="Average Price by Location")
    col1.plotly_chart(fig1, use_container_width=True)

    # Graph 2: Average Price by Property Type
    avg_price_property = filtered_data.groupby('property_type')['monthly_rent'].mean().reset_index()
    avg_price_property['monthly_rent'] = avg_price_property['monthly_rent'].round(2)  # Round to 2 decimal places
    fig2 = px.bar(avg_price_property, x='property_type', y='monthly_rent', color='property_type', title="Average Price by Property Type")
    col2.plotly_chart(fig2, use_container_width=True)

Overwriting dashboard.py


In [ ]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import pickle

st.title("Rent Pricing at Kuala Lumpur and Selangor")

# Load and preprocess dataset
@st.cache_data
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/FYP B IZZA/mudah-apartment-kl-selangor_cleaned.csv')  # Replace with your dataset file
    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()

    # Clean 'monthly_rent' column (removes RM, per month, and spaces)
    df['monthly_rent'] = (
        df['monthly_rent']
        .str.replace('RM', '', regex=False)           # Remove RM
        .str.replace('per month', '', regex=False)    # Remove 'per month'
        .str.replace(' ', '', regex=False)            # Remove spaces between numbers
        .apply(pd.to_numeric, errors='coerce')         # Convert to float, coerce errors to NaN
    )

    # Ensure numeric columns are properly converted to numeric, coercing errors to NaN
    numeric_columns = ['rooms', 'bathroom', 'parking']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Use pd.to_numeric to handle non-numeric values

    # Drop rows with NaN values in numeric columns if necessary (e.g., for prediction)
    df = df.dropna(subset=numeric_columns + ['monthly_rent'])

    return df

df = load_data()

# Check if required columns exist
required_columns = ['location', 'property_type', 'rooms', 'bathroom', 'parking', 'monthly_rent']
if not all(col in df.columns for col in required_columns):
    st.error(f"The dataset must contain the following columns: {required_columns}")
    st.stop()

# Load trained model
@st.cache_resource
def load_model():
    with open('/content/drive/MyDrive/FYP B IZZA/best_model_pipeline.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

model = load_model()

# Sidebar for user input
st.sidebar.title('Input Parameters for Prediction')
st.sidebar.header('Prediction Inputs')

# Dynamically populate dropdowns and sliders
location = st.sidebar.selectbox('Select Location', df['location'].unique())
property_type = st.sidebar.selectbox('Select Property Type', df['property_type'].unique())
bedrooms = st.sidebar.slider('Number of Bedrooms', min_value=int(df['rooms'].min()), max_value=int(df['rooms'].max()), step=1)
bathrooms = st.sidebar.slider('Number of Bathrooms', min_value=int(df['bathroom'].min()), max_value=int(df['bathroom'].max()), step=1)
parking = st.sidebar.slider('Number of Parking Spaces', min_value=int(df['parking'].min()), max_value=int(df['parking'].max()), step=1)

# Prepare input data for prediction
input_data = pd.DataFrame({
    'location': [location],
    'property_type': [property_type],
    'rooms': [bedrooms],
    'bathroom': [bathrooms],
    'parking': [parking],
})

# Prediction button
if st.sidebar.button('Predict'):
    try:
        prediction = model.predict(input_data)
        st.sidebar.success(f"Predicted Rent: RM{prediction[0]:,.2f}")
    except Exception as e:
        st.sidebar.error(f"Prediction failed: {e}")

# Interactive Graphs Section
st.header("Interactive Graphs")
st.sidebar.header('Graph Parameters')

# Filter dataset based on user selection
selected_locations = st.sidebar.multiselect('Select Locations', df['location'].unique(), default=df['location'].unique())
selected_property_types = st.sidebar.multiselect('Select Property Types', df['property_type'].unique(), default=df['property_type'].unique())

filtered_data = df[
    (df['location'].isin(selected_locations)) &
    (df['property_type'].isin(selected_property_types))
]

# Visualization Columns
col1, col2 = st.columns(2)

# Graph 1: Average Price by Location
if not filtered_data.empty:
    avg_price_location = filtered_data.groupby('location')['monthly_rent'].mean().reset_index()
    fig1 = px.bar(avg_price_location, x='location', y='monthly_rent', color='location', title="Average Price by Location")
    col1.plotly_chart(fig1, use_container_width=True)

# Graph 2: Average Price by Property Type
if not filtered_data.empty:
    avg_price_property = filtered_data.groupby('property_type')['monthly_rent'].mean().reset_index()
    fig2 = px.bar(avg_price_property, x='property_type', y='monthly_rent', color='property_type', title="Average Price by Property Type")
    col2.plotly_chart(fig2, use_container_width=True)

# Scatter plots for additional insights
col3, col4 = st.columns(2)
if not filtered_data.empty:
    fig3 = px.scatter(filtered_data, x='rooms', y='monthly_rent', color='location', title="Bedrooms vs. Rent")
    col3.plotly_chart(fig3, use_container_width=True)

    fig4 = px.scatter(filtered_data, x='bathroom', y='monthly_rent', color='location', title="Bathrooms vs. Rent")
    col4.plotly_chart(fig4, use_container_width=True)


Overwriting dashboard.py
